In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from utils import *
from joblib import dump, load
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamblock/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adamblock/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/adamblock/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adamblock/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adamblock/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/adamblock/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
corpus_specific_stopwords

{'change',
 'climate',
 'dont',
 'global',
 'he',
 'im',
 'say',
 'thats',
 'thing',
 'warming'}

# Getting and Preprocessing Data

In [4]:
PARENT = "../TelevisionNews/"

In [5]:
cnn, fox, msnbc = get_data(PARENT)

In [6]:
cnn["cleanSnip"] = cnn["Snippet"].apply(lambda x: preprocessing(x, stop_words,True))
fox["cleanSnip"] = fox["Snippet"].apply(lambda x: preprocessing(x, stop_words,True))
msnbc["cleanSnip"] = msnbc["Snippet"].apply(lambda x: preprocessing(x, stop_words,True))

In [7]:
all_shows = pd.concat([cnn,fox,msnbc])

# TFIDF and NMF

In [8]:
def display_topics(model, feature_names, no_top_words=5):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                          for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [9]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df =25, max_features=5000, use_idf=True)
tfidf = tfidf_vectorizer.fit_transform(all_shows['cleanSnip'])
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [16]:
num_topics = 10

In [17]:
doc_term_matrix_tfidf = pd.DataFrame(tfidf.toarray(), columns=list(tfidf_feature_names))

In [18]:
nmf = NMF(n_components=num_topics).fit(tfidf)

/Users/adamblock/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1076: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"


In [19]:
display_topics(nmf, tfidf_feature_names,10)

Topic 0:
new year one right time well would back want like
Topic 1:
gas greenhouse emission cut growing fuel reduce potential keep half
Topic 2:
going get touched everywhere pot impacted specie melting go youre
Topic 3:
president trump obama house said donald white paris administration hoax
Topic 4:
talk scientist sensor road tree network car internet everything call
Topic 5:
issue care health like immigration republican democrat important gun voter
Topic 6:
believe know science real scientist said human doesnt man question
Topic 7:
people american want lot like make young country environment solution
Topic 8:
think he would going something lot well there important really
Topic 9:
state world united country threat nation leader china america around


In [15]:
dump(nmf, f"./models/NMF_all_{num_topics}_topics.joblib")

['./models/NMF_all_16_topics.joblib']

# LDA?

In [25]:
tf_vectorizer = CountVectorizer(max_df=0.9, min_df=25, max_features=5000)
tf = tf_vectorizer.fit_transform(all_shows['cleanSnip'].values.astype('U'))
tf_feature_names = tf_vectorizer.get_feature_names()

In [26]:
doc_term_matrix = pd.DataFrame(tf.toarray(), columns=list(tf_feature_names))

,abandoned,abby,abc,ability,able,abortion,abroad,absence,absent,absolute,...,youll,young,younger,youre,youth,youtube,youve,zero,zinke,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71593,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71594,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
71595,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, learning_method='online', max_iter=500, random_state=0).fit(tf)

KeyboardInterrupt: 